### We need to select the middle position of cylinder cases and augment it
### Observation window [100*64] in middle
### [0,10,20,30,40,50] --6
### 36 x 15 = 540

In [ ]:
import numpy as np

In [ ]:
cases_36 = np.load("/root/autodl-tmp/HSPS/IFC/Dataset/36_cases/36_cases_0321.npz")

In [ ]:
cases_36.files

In [ ]:
cases_36["Condition"].shape

In [ ]:
cases_36["Condition"][0]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(cases_36["tensor"][0,0,:,:],cmap="jet")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import rotate

# 假设你的大矩形数据是 [100, 128, 512]，其中 100 是时间
# 这里我们模拟一个随机矩阵来代表这个数据
x_center, y_center = 256, 64  # (x, y) 是中心坐标
observation_width, observation_height = 100, 64
# 获取黄色区域的边界
x_min = x_center - observation_width // 2
x_max = x_center + observation_width // 2
y_min = y_center - observation_height // 2
y_max = y_center + observation_height // 2

def Rotate_cases(data,angle=-10):
    '''
        逆时针
        input: data:[t,128,512]
        output: ob_data:[t,64,100]
    '''
    # 中间黄色区域的中心位置，假设黄色区域大小是 100x64
    
   
    # 复制原始数据，保留黄色区域
    rotated_data = data.copy()

    for t in range(data.shape[0]):  # 逐个时间步
        # 逐个处理每个时间步的数据（128x512）
        image = data[t]  # 取出某个时间步的数据
        
        # 对外部区域进行旋转
        rotated_image = rotate(image, angle, reshape=False, mode='constant')

        # 更新旋转后的数据
        rotated_data[t] = rotated_image
    
    extracted_region = rotated_data[:, y_min:y_max, x_min:x_max]
    return extracted_region

In [ ]:
Angle_list= [0,-10,-20,-30,-40,-50]

In [ ]:
## Augment cases

data = cases_36["tensor"]
case_numbers = cases_36["tensor"].shape[0]

Augment_tensor = np.zeros((36*6,80,64,100))
Augment_conditions = np.zeros((36*6,4))
case_n =0
for case_number in range(case_numbers):
   data = cases_36["tensor"][case_number]
   case_condition = cases_36["Condition"][case_number][-3:]
   
   for angle in Angle_list:
      
      Augment_conditions[case_n,-3:] = case_condition 
      Augment_conditions[case_n,0] = angle
      
      Augment_tensor[case_n] = Rotate_cases(data,angle)
      case_n = case_n+1
      print("case_n",case_n)

In [70]:
import imageio
import numpy as np
import matplotlib.pyplot as plt

# Assuming Augment_tensor has dimensions [time_steps, height, width] for simplicity
# If your actual dimensions include more (like batches), you'll need to adjust the indexing accordingly.

# Set the filename for the GIF
filename = 'augmented_data.mp4'

# Create a writer object from imageio to write GIF with a specific duration per frame
writer = imageio.get_writer(filename, fps=5)  # frames per second
b = -1
# Iterate through each time step
for t in range(Augment_tensor.shape[1]):
    # Create a figure
    fig, ax = plt.subplots()
    # Display the image
    im = ax.imshow(Augment_tensor[b,t, :, :], cmap='viridis')
    ax.set_title(f"Angle: {Augment_conditions[b, 0]},D:{2*Augment_conditions[b, 1]*0.00195:.2f}(m)\n\
                 Speed:{Augment_conditions[b,2]}(m/s),Time:{(Augment_conditions[b,3]+t)*0.05:.3f}(s)")
    ax.axis('off')  # Turn off axis

    # Save the plot to a buffer instead of displaying
    plt.savefig('temp.png', bbox_inches='tight',dpi=300)
    plt.close()

    # Read the image back in
    image = imageio.imread('temp.png')
    # Append the image to the GIF
    writer.append_data(image)

# Close the writer to finish writing the GIF
writer.close()

# Remove the temporary image file
import os
os.remove('temp.png')

print(f"GIF saved as {filename}")


/tmp/ipykernel_5915/1549858860.py:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread('temp.png')
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1548, 1130) to (1552, 1136) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


GIF saved as augmented_data.mp4


In [71]:
np.savez("216_cases_augment_0321.npz", tensor=Augment_tensor, Condition=Augment_conditions)